<a href="https://colab.research.google.com/github/kcalizadeh/phil_nlp/blob/master/dash_app_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Mounting Drive

In [6]:
# this cell mounts drive, sets the correct directory, then imports all functions
# and relevant libraries via the functions.py file
from google.colab import drive
import sys

# install relevent libraries not included with colab
!pip install lime
!pip install symspellpy
!pip install jupyter-dash
!pip install dash-bootstrap-components


drive.mount('/gdrive',force_remount=True)

drive_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP'

sys.path.append(drive_path)

     |████████████████████████████████| 276kB 7.1MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283846 sha256=3ed22479e7648de2cab7fb5923c86bf9cd1280256810168a153274c562675990
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime
     |████████████████████████████████| 2.6MB 4.6MB/s 
Mounted at /gdrive


In [8]:
from functions import *
%load_ext autoreload
%autoreload 2

np.random.seed(17)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [231]:
class Padder(BaseEstimator, TransformerMixin):
    def __init__(self, maxlen=500):
        self.maxlen = maxlen
        self.max_index = None
        
    def fit(self, X, y=None):
        self.max_index = pad_sequences(X, maxlen=self.maxlen).max()
        return self
    
    def transform(self, X, y=None):
        X = pad_sequences(X, maxlen=self.maxlen)
        # X[X > self.max_index] = 0
        return X

padder = Padder(450)

In [233]:
class TextsToSequences(BaseEstimator, TransformerMixin):
    def __init__(self,  tokenizer):
        self.tokenizer = tokenizer
        
    def fit(self, texts, y=None):
        return self
    
    def transform(self, texts, y=None):
        return np.array(self.tokenizer.texts_to_sequences(texts))
        
sequencer = TextsToSequences(tokenizer)

In [236]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

# Load Data
df = pd.read_csv('/gdrive/MyDrive/Colab_Projects/Phil_NLP/phil_nlp.csv')

model_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP/checkpoints/NN_weights_epoch:07_0.7678.hdf5'
model = load_model(model_path)

with open('/gdrive/MyDrive/Colab_Projects/Phil_NLP/baseline_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# set up classification explanation pipeline
padder = Padder(450)
sequencer = TextsToSequences(tokenizer)
pipeline = make_pipeline(sequencer, padder, model)

# set up labels
school_label_dict = {'analytic': 0,
 'aristotle': 1,
 'capitalism': 2,
 'communism': 3,
 'continental': 4,
 'empiricism': 5,
 'german_idealism': 6,
 'phenomenology': 7,
 'plato': 8,
 'rationalism': 9}
flipped_dict = {value:key for key, value in school_label_dict.items()}

# search bar object
search_bar = html.Div(id="search-bar-container", children=
    [
        dbc.Input(id="search-bar", placeholder="enter text to classify", type="text"),
        dbc.Button("SUBMIT", id="search-bar-submit-button", color="primary", className="mr-1", n_clicks=0)
    ])


# the app itself
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Text Classification"),
    search_bar,
    html.Div(id="search-bar-output", children=[]),

])


# callback for search bar
@app.callback(Output(component_id="search-bar-output", component_property="children"),
              [Input(component_id="search-bar-submit-button", component_property="n_clicks")],
              [State(component_id="search-bar", component_property="value")])
def generate_explainer_html(n_clicks, text):
    empty_obj = html.Iframe(
        srcDoc='''<div>Enter input text to see LIME explanations.</div>''',
        width='100%',
        height='100px',
        style={'border': '2px #d3d3d3 solid'},
        hidden=True,
    )
    if n_clicks < 1 or text == '':
      return empty_obj
    else:
      explainer = lime_text.LimeTextExplainer(class_names=list(school_label_dict.keys()))
      exp = explainer.explain_instance(text, pipeline.predict, num_features=10, labels=[0,1,2,3,4,5,6,7,8,9])
      obj = html.Iframe(
          srcDoc=exp.as_html(),
          width='100%',
          height='800px',
          style={'border': '2px #d3d3d3 solid'},
      )
      return obj

# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>

In [228]:
from tensorflow.keras.models import load_model


In [103]:
model_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP/checkpoints/NN_weights_epoch:07_0.7678.hdf5'
model = load_model(model_path)



In [104]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         10844288  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                35800     
_________________________________________________________________
dense (Dense)                (None, 25)                1275      
_________________________________________________________________
dropout (Dropout)            (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                260       
Total params: 10,881,623
Trainable params: 10,881,623
Non-trainable params: 0
_________________________________________________________________


In [105]:
with open('/gdrive/MyDrive/Colab_Projects/Phil_NLP/baseline_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [151]:
to_classify = 'Knowledge of the Idea of the absolute ethical order depends entirely on the establishment of perfect adequacy between intuition and concept, because the Idea itself is nothing other than the identity of the two. But if this identity is to be actually known, it must be thought as a made adequacy.'

In [194]:
pipeline = make_pipeline(sequencer, padder, model)

In [152]:
pd.Series(to_classify)

0    Knowledge of the Idea of the absolute ethical ...
dtype: object

In [153]:
tokenized = tokenizer.texts_to_sequences(pd.Series(to_classify))

In [154]:
padded = sequence.pad_sequences(tokenized, maxlen=450)

In [198]:
flipped_dict[pipeline.predict(pd.Series(to_classify)).argmax()]

'german_idealism'

In [155]:
prediction_num = model.predict(padded, verbose=1)

1/1 [==============================] - 0s 27ms/step


In [156]:
prediction_num.argmax()

6

In [157]:
flipped_dict[prediction_num.argmax()]

'german_idealism'

In [172]:
list(school_label_dict.keys())

['analytic',
 'aristotle',
 'capitalism',
 'communism',
 'continental',
 'empiricism',
 'german_idealism',
 'phenomenology',
 'plato',
 'rationalism']

In [209]:
to_classify = """Hi Michelle and Kourosh-

 

Happy New Year to you both!

 

Was wondering if we could schedule a short touch base in the next few weeks? No concerns—rather—just want to get some feedback on Cole’s progress.

 

Could you suggest a few times that could work for you both?

 

thanks"""

In [ ]:
explainer = lime_text.LimeTextExplainer(class_names=list(school_label_dict.keys()))
exp = explainer.explain_instance(to_classify, pipeline.predict, num_features=10, labels=[0,1,2,3,4,5,6,7,8,9])

exp.show_in_notebook(text=True)

In [ ]:
for i in prediction_num.argmax(axis=1):
  print(flipped_dict[i])

In [93]:
school_label_dict = {'analytic': 0,
 'aristotle': 1,
 'capitalism': 2,
 'communism': 3,
 'continental': 4,
 'empiricism': 5,
 'german_idealism': 6,
 'phenomenology': 7,
 'plato': 8,
 'rationalism': 9}
flipped_dict = {value:key for key, value in school_label_dict.items()}

In [188]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.pipeline import TransformerMixin
from sklearn.base import BaseEstimator

class TextsToSequences(BaseEstimator, TransformerMixin):
    """ Sklearn transformer to convert texts to indices list 
    (e.g. [["the cute cat"], ["the dog"]] -> [[1, 2, 3], [1, 4]])"""
    def __init__(self,  tokenizer, **kwargs):
        self.tokenizer = tokenizer
        
    def fit(self, texts, y=None):
        return self
    
    def transform(self, texts, y=None):
        return np.array(self.tokenizer.texts_to_sequences(texts))
        
sequencer = TextsToSequences(tokenizer, num_words=450)

In [192]:
class Padder(BaseEstimator, TransformerMixin):
    """ Pad and crop uneven lists to the same length. 
    Only the end of lists longer than the maxlen attribute are
    kept, and lists shorter than maxlen are left-padded with zeros
    
    Attributes
    ----------
    maxlen: int
        sizes of sequences after padding
    max_index: int
        maximum index known by the Padder, if a higher index is met during 
        transform it is transformed to a 0
    """
    def __init__(self, maxlen=500):
        self.maxlen = maxlen
        self.max_index = None
        
    def fit(self, X, y=None):
        self.max_index = pad_sequences(X, maxlen=self.maxlen).max()
        return self
    
    def transform(self, X, y=None):
        X = pad_sequences(X, maxlen=self.maxlen)
        # X[X > self.max_index] = 0
        return X

padder = Padder(450)